In [ ]:
# Import statements
import numpy as np
import matplotlib.pyplot as pt
import glob
import time
import os
import shutil
import pathlib
import matplotlib

from ase.io import read
from pyiron import Project, ase_to_pyiron

from molmod.units import *
from molmod.constants import *

from collections import namedtuple, Counter
from dataclasses import dataclass # replaces namedtuple with mutable attributes
%matplotlib inline

matplotlib.rcParams.update({'font.size': 10})


In [ ]:
# Create project for rotational barriers characterization
pr = Project('barriers')

## Job Functions

In [ ]:
# g16 job function
def g16_job(pr, structure, name, jobtype='sp', lot='B3LYP', basis_set='6-311++G(d,p)', settings={'EmpiricalDispersion':['GD3'], 'int':['grid=ultrafine'], 'scf':['tight','maxcycle=5000']}, suffix=None, cores=9, run_time=60*60):
    job = pr.create_job(pr.job_type.Gaussian, name, delete_existing_job=True)
    job.structure = structure
    job.input['jobtype'] = jobtype
    job.input['lot'] = lot
    job.input['basis_set'] = basis_set
    
    if not settings is None:
        job.input['settings'] = settings
        
    if suffix is not None:
        job.input['suffix'] = suffix
        
    job.executable.version = '2019_mpi' # to work in kirlia
    
    job.server.queue = 'kirlia'
    job.server.cores = cores
    job.server.run_time = run_time # in seconds
    
    job.run()

In [ ]:
# g16 job function
def g16_gic_job(pr, structure, name, jobtype='opt', lot='B3LYP', basis_set='6-311++G(d,p)', settings={'EmpiricalDispersion':['GD3'], 'int':['grid=ultrafine'], 'scf':['tight','maxcycle=5000'], 'geom':['addgic'], 'nosymm':[]}, suffix=None, run_time=2*60*60):
    job = pr.create_job(pr.job_type.Gaussian, name, delete_existing_job=True)
    job.structure = structure
    job.input['jobtype'] = jobtype
    job.input['lot'] = lot
    job.input['basis_set'] = basis_set
    
    if not settings is None:
        job.input['settings'] = settings
        
    if suffix is not None:
        job.input['suffix'] = suffix
        
    job.executable.version = '2019_mpi' # to work in kirlia
    
    job.server.queue = 'kirlia'
    job.server.cores = 9
    job.server.run_time = run_time # in seconds
    
    job.run()

In [ ]:
# yaff job function
def yaff_opt_job(pr, name, structure, ffs, ffpars):
    job = pr.create_job(pr.job_type.Yaff, name, delete_existing_job=True)
    
    job.calc_minimize(max_iter=10000)

    job.input['ffpars'] = ffpars
    job.structure = structure
    job.ffatypes = ffs.ffatypes
    job.ffatype_ids = ffs.ffatype_ids
    job.bonds = ffs.bonds
    
    job.executable.version = '2020'
    job.server.queue = 'slaking'
    job.server.cores = 1
    job.server.run_time = 5*60*60 # in seconds

    job.run()


In [ ]:
# yaff job function
def yaff_scan_job(pr, name, sinfo, ffpars):
    job = pr.create_job(pr.job_type.Yaff, name, delete_existing_job=True)
    job.structure = sinfo.ff.structure
    job.input['ffpars'] = ffpars # separate to allow for varying ffpars
    
    job.ffatypes = sinfo.ff.ffatypes
    job.ffatype_ids = sinfo.ff.ffatype_ids
    job.bonds = sinfo.ff.bonds

    job.calc_scan(np.arange(sinfo.range_min, sinfo.range_max+sinfo.range_step, sinfo.range_step), adapt_structure=sinfo.adapt_structure)
    job.executable.version = '2020'
    job.server.queue = 'slaking'
    job.server.cores = 1
    job.server.run_time = 15*60 # in seconds   
    job.run()


In [ ]:
# yaff job function
def yaff_scan_structures_job(pr, name, sinfo, ffpars, structures):
    job = pr.create_job(pr.job_type.Yaff, name, delete_existing_job=True)
    job.structure = sinfo.ff.structure
    job.input['ffpars'] = ffpars # separate to allow for varying ffpars
    
    job.ffatypes = sinfo.ff.ffatypes
    job.ffatype_ids = sinfo.ff.ffatype_ids
    job.bonds = sinfo.ff.bonds

    job.calc_scan(np.arange(sinfo.range_min, sinfo.range_max+sinfo.range_step, sinfo.range_step),structures=structures)
    job.executable.version = '2020'
    job.server.queue = 'slaking'
    job.server.cores = 1
    job.server.run_time = 15*60 # in seconds   
    job.run()

## AI scans

### Functions

In [ ]:
'''@dataclass
class ff_object:
    structure: object
    ffpars: object
    ffpars_nodih: object
    ffpars_polysix: object
    ffatypes: object
    ffatype_ids: object
    bonds : object'''
        
        
class FFobject(object):
    def __init__(self,block_name,structure,ffatypes,ffatype_ids,bonds,fnames,fn_ai,ffpars_nodih=None,ffpars_polysix=None):
        self.block_name = block_name
        self.structure = structure
        self.ffatypes = ffatypes
        self.ffatype_ids = ffatype_ids
        self.bonds = bonds
        self.fnames = fnames
        self.ffpars = self._set_ffpars()
        self.fn_ai = fn_ai
        self.ffpars_nodih = ffpars_nodih
        self.ffpars_polysix = ffpars_polysix
        
    def _set_ffpars(self):
        ffpars = ''
        for fn in self.fnames:
            with open(fn,'r') as f:
                ffpars+=f.read()
        return ffpars

In [ ]:
# Structure info and rotation grid
class Sinfo(object):
    def __init__(self,pr,ff,indices,mask=None,range_min=0,range_max=90,range_step=1,degeneracy=None, fit_indices=None):
        self.pr = pr
        self.ff = FFobject(ff.block_name,ff.structure,ff.ffatypes,ff.ffatype_ids,ff.bonds,ff.fnames,ff.fn_ai,ffpars_nodih=ff.ffpars_nodih,ffpars_polysix=ff.ffpars_polysix) # create new object to avoid overwritten when using the same FFObject for difference sinfos
        self.indices = indices
        self.mask = mask
        self.mask_indices = None 
        self.range_min = range_min
        self.range_max = range_max
        self.range_step = range_step
        self.degeneracy = degeneracy
        
        self.fit_indices = fit_indices
        
        self._set_mask_indices()
        self.adapt_structure = self._set_adapt_structure()
        
        
    # Define function to identify mask for rotation
    @staticmethod
    def get_mask(structure,indices):
        # Assume that we can define mask by taking all atoms in the direction of the dihedral
        ref_vector = structure.positions[indices[2]] - structure.positions[indices[1]]
        mask = np.array([np.dot(pos - structure.positions[indices[1]], ref_vector) >= 0 for pos in structure.positions],dtype=np.int)
        return mask
        
    def _set_mask_indices(self):
        if self.mask is None:
            self.mask = self.get_mask(self.ff.structure,self.indices)
        else:
            if len(self.mask)<len(self.ff.structure):
                self.mask_indices = copy.copy(self.mask)
                self.mask = None
            else:
                assert len(self.mask)==len(self.ff.structure)
                self.mask_indices=None
                
    def _set_adapt_structure(self):
        
        def adapt_structure(structure,angle,idx=self.indices,mask=self.mask,indices=self.mask_indices):
            new_structure = structure.copy()
            new_structure.set_dihedral(*idx,angle=angle,mask=mask,indices=indices) # angle in degrees
            return new_structure
        
        return adapt_structure
                
    def get_sub_project_name(self,key,suffix=None):
        if suffix is not None:
            return 'barriers/{}/{}_{}_{}'.format(self.pr.name,key,"_".join([str(i) for i in self.indices]),suffix)
        else:
            return 'barriers/{}/{}_{}'.format(self.pr.name,key,"_".join([str(i) for i in self.indices]))

In [ ]:
# ICs check (only dihedral can change)

from yaff import *
from yaff.log import *
import copy

log.set_level(0)

def get_ICs(system, ics):
    num = 0
    ICl = InternalCoordinateList(DeltaList(system))
    for ic in ics:
        ICl.add_ic(ic)
        num+=1
    ICl.dlist.forward()
    ICl.forward()

    values = np.zeros(num)
    for n in range(num):
        values[n] = ICl.ictab[n][-2]

    return values

# Define function to read dihedral ffatype sets to see whether they are correctly removed
def read_dihedral_terms(ffpars):
    dih_set = []
    for line in ffpars.split('\n'):
        if line.startswith('TORSION:PARS'):
            l = line.split()
            dih_set.append(tuple((l[1],l[2],l[3],l[4])))
    return set(dih_set)



def write_ffpars_nodih(name,sinfo):
    dih_values = []
    dihedral_set = read_dihedral_terms(sinfo.ff.ffpars)

    grid = np.arange(sinfo.range_min, sinfo.range_max+sinfo.range_step, sinfo.range_step)
    for n,val in enumerate(grid):
        adapted_structure = sinfo.adapt_structure(sinfo.ff.structure,val)
        system = System(sinfo.ff.structure.numbers, adapted_structure.positions, ffatypes=sinfo.ff.ffatypes, ffatype_ids=sinfo.ff.ffatype_ids, bonds=sinfo.ff.bonds)

        dihs = []
        dihs_angle = []
        for dih in system.iter_dihedrals():
            dihs.append([system.ffatypes[system.ffatype_ids[i]] for i in dih])
            dihs_angle.append(DihedAngle(*dih))

        dih_values.append(np.round(np.array(get_ICs(system, dihs_angle)),8))

    dih_values = np.asarray(dih_values)

    print("{} ICs\n".format(name))

    # print out those that change during trajectory
    dihedral_deviations = np.zeros(dih_values.shape[1])    
    for n in range(len(dihedral_deviations)):
        dihedral_deviations[n] = np.average(np.arccos(np.cos(dih_values[:,n]-dih_values[-1,n])))

    print("Dihedrals:")
    degeneracy = 0
    for n in range(len(dihedral_deviations)):
        if dihedral_deviations[n] > 5e-1:
            degeneracy += 1
            print(dihs[n])#, dihedral_deviations[n])
            if tuple(dihs[n]) in dihedral_set:
                dihedral_set.remove(tuple(dihs[n]))
            elif tuple(dihs[n][::-1]) in dihedral_set:
                dihedral_set.remove(tuple(dihs[n][::-1]))
                
    sinfo.degeneracy = degeneracy
    
    # Making adapted force field parameter file
    ffpars_reduced = ''
    for line in sinfo.ff.ffpars.split('\n'):
        if line.startswith('TORSION:PARS'):
            l = line.split()
            dih = tuple((l[1],l[2],l[3],l[4]))
            if dih in dihedral_set:
                ffpars_reduced+=line+'\n'
            else:
                print('Removed the following line: ', line)
        else:
            ffpars_reduced+=line+'\n'
            
    sinfo.ff.ffpars_nodih = ffpars_reduced
    print('\n\n')
                

def plot_IC_variations(name,sinfo):
    print(sinfo.ff.block_name)
    bond_values = []
    bend_values = []
    dih_values = []
  
    dihedral_set = read_dihedral_terms(sinfo.ff.ffpars)
    if sinfo.ff.structure.cell.volume >0:
        system = System(sinfo.ff.structure.numbers, sinfo.ff.structure.positions, ffatypes=sinfo.ff.ffatypes, ffatype_ids=sinfo.ff.ffatype_ids, bonds=sinfo.ff.bonds, rvecs=sinfo.ff.structure.cell.array)
    else:
        system = System(sinfo.ff.structure.numbers, sinfo.ff.structure.positions, ffatypes=sinfo.ff.ffatypes, ffatype_ids=sinfo.ff.ffatype_ids, bonds=sinfo.ff.bonds)
    yaff_ff = ForceField.generate(system, sinfo.ff.fnames, rcut=12.0*angstrom, alpha_scale=3.2, gcut_scale=1.5, smooth_ei=True, tailcorrections=True)

    grid = np.arange(sinfo.range_min, sinfo.range_max+sinfo.range_step, sinfo.range_step)
    for n,val in enumerate(grid):
        adapted_structure = sinfo.adapt_structure(sinfo.ff.structure,val)
        yaff_ff.update_pos(adapted_structure.positions)
        #system = System(sinfo.ff.structure.numbers, adapted_structure.positions, ffatypes=sinfo.ff.ffatypes, ffatype_ids=sinfo.ff.ffatype_ids, bonds=sinfo.ff.bonds)

        dihs = []
        dihs_angle = []
        for dih in yaff_ff.system.iter_dihedrals():
            dihs.append([yaff_ff.system.ffatypes[yaff_ff.system.ffatype_ids[i]] for i in dih])
            dihs_angle.append(DihedAngle(*dih))


        bends = []
        bend_angles = []
        for bend in system.iter_angles():
            bends.append([yaff_ff.system.ffatypes[yaff_ff.system.ffatype_ids[i]] for i in bend])
            bend_angles.append(BendAngle(*bend))

        bonds = []
        bond_lengths = []
        for bond in system.iter_bonds():
            bonds.append([yaff_ff.system.ffatypes[yaff_ff.system.ffatype_ids[i]] for i in bond])
            bond_lengths.append(Bond(*bond))

        bond_values.append(np.array(get_ICs(yaff_ff.system, bond_lengths)))
        bend_values.append(np.array(get_ICs(yaff_ff.system, bend_angles)))
        dih_values.append(np.round(np.array(get_ICs(yaff_ff.system, dihs_angle)),8))

    bond_values = np.asarray(bond_values)
    bend_values = np.asarray(bend_values)
    dih_values = np.asarray(dih_values)

    print("{} ICs\n".format(name))

    # print out those that change during trajectory
    
    #indx = np.where(bond_values[0,:]>10)[0]
    #for i in indx:
    #    print(bonds[i],bond_values[0,i], list(system.iter_bonds())[i])
    
    
    bond_deviations = np.zeros(bond_values.shape[1])
    for n in range(len(bond_deviations)):
        bond_deviations[n] = np.sum(np.abs(bond_values[:,n] - np.average(bond_values[:,n])))

    print("Bonds:")
    for n in range(len(bond_deviations)):
        if not bond_deviations[n] < 1e-5:
            print(bonds[n], bond_deviations[n])

    bend_deviations = np.zeros(bend_values.shape[1])
    for n in range(len(bend_deviations)):
        bend_deviations[n] = np.sum(np.abs(bend_values[:,n] - np.average(bend_values[:,n])))

    print("Bends:")
    for n in range(len(bend_deviations)):
        if not bend_deviations[n] < 1e-5:
            print(bends[n])#, bend_deviations[n])

    dihedral_deviations = np.zeros(dih_values.shape[1])    
    for n in range(len(dihedral_deviations)):
        dihedral_deviations[n] = np.average(np.arccos(np.cos(dih_values[:,n]-dih_values[-1,n])))

    print("Dihedrals:")
    for n in range(len(dihedral_deviations)):
        if not dihedral_deviations[n] < 2e-1:
            print(dihs[n], dihedral_deviations[n])
            try:
                assert tuple(dihs[n]) in dihedral_set or tuple(dihs[n][::-1]) in dihedral_set
            except AssertionError:
                print('There was no FF term for the following dihedral: ', tuple(dihs[n]))
                
    print("\n\n")


    # Plot trajectory of ICs
    fig,ax = pt.subplots(1,3,figsize=(15,10))
    for n in range(bond_values.shape[1]):
        ax[0].plot(grid,bond_values[:,n])
    ax[0].set_title('Bonds')

    for n in range(bend_values.shape[1]):
        ax[1].plot(grid,np.rad2deg(bend_values[:,n]))
    ax[1].set_title('Bends')

    for n in range(dih_values.shape[1]):
        if not dihedral_deviations[n] < 2e-1:
            ax[2].plot(grid,np.rad2deg(dih_values[:,n]),label=str(dihs[n]))
    ax[2].legend(bbox_to_anchor=(1.1,.5), loc='center left',frameon=False)
    ax[2].set_title('Dihedrals')
    pt.show()

def compare_structures(name,pr,ff,structures,log_bonds=True,log_bends=True,log_dihedrals=True,save_png=False):
    bond_values = []
    bend_values = []
    dih_values = []
  

    for n,structure in enumerate(structures.values()):
        system = System(structure.numbers, structure.positions, ffatypes=ff.ffatypes, ffatype_ids=ff.ffatype_ids, bonds=ff.bonds)

        dihs = []
        dihs_angle = []
        for dih in system.iter_dihedrals():
            dihs.append([system.ffatypes[system.ffatype_ids[i]] for i in dih])
            dihs_angle.append(DihedAngle(*dih))


        bends = []
        bend_angles = []
        for bend in system.iter_angles():
            bends.append([system.ffatypes[system.ffatype_ids[i]] for i in bend])
            bend_angles.append(BendAngle(*bend))

        bonds = []
        bond_lengths = []
        for bond in system.iter_bonds():
            bonds.append([system.ffatypes[system.ffatype_ids[i]] for i in bond])
            bond_lengths.append(Bond(*bond))

        bond_values.append(np.array(get_ICs(system, bond_lengths)))
        bend_values.append(np.array(get_ICs(system, bend_angles)))
        dih_values.append(np.round(np.array(get_ICs(system, dihs_angle)),8))

    bond_values = np.asarray(bond_values)
    bend_values = np.asarray(bend_values)
    dih_values = np.asarray(dih_values)

    print("{} ICs\n".format(name))

    # print out those that have a large difference between reference (-1) and latest FF calculation (-2) 
    bond_deviations = np.zeros(bond_values.shape[1])
    for n in range(len(bond_deviations)):
        bond_deviations[n] = np.sum(np.abs(bond_values[-2,n] - bond_values[-1,n]))
        
    bond_deviations_old = np.zeros(bond_values.shape[1])
    if len(structures)==3:
        for n in range(len(bond_deviations_old)):
            bond_deviations_old[n] = np.sum(np.abs(bond_values[0,n] - bond_values[-1,n]))

    if log_bonds:
        print("Bonds:")
        for n in range(len(bond_deviations)):
            if not bond_deviations[n] < 1e-2:
                print(bonds[n], bond_deviations[n])

    bend_deviations = np.zeros(bend_values.shape[1])
    for n in range(len(bend_deviations)):
        bend_deviations[n] = np.sum(np.abs(bend_values[-2,n] - bend_values[-1,n]))
        
    bend_deviations_old = np.zeros(bend_values.shape[1])
    if len(structures)==3:
        for n in range(len(bend_deviations_old)):
            bend_deviations_old[n] = np.sum(np.abs(bend_values[0,n] - bend_values[-1,n]))

    if log_bends:
        print("Bends:")
        for n in range(len(bend_deviations)):
            if not bend_deviations[n] < 2*deg:
                print(bends[n], np.rad2deg(bend_deviations[n]))

    dihedral_deviations = np.zeros(dih_values.shape[1])
    for n in range(len(dihedral_deviations)):
        dihedral_deviations[n] = np.average(np.arccos(np.cos(dih_values[-2,n]-dih_values[-1,n])))
        
        
    dihedral_deviations_old = np.zeros(dih_values.shape[1])
    if len(structures)==3:
        for n in range(len(dihedral_deviations_old)):
            dihedral_deviations_old[n] = np.average(np.arccos(np.cos(dih_values[0,n]-dih_values[-1,n])))

    if log_dihedrals:
        uniq_dih_axes = []
        print("Dihedrals:")
        for n in range(len(dihedral_deviations)):
            if not dihedral_deviations[n] < 10*deg:
                if dihs[n][1:3] not in uniq_dih_axes and dihs[n][2:0:-1] not in uniq_dih_axes:
                    uniq_dih_axes.append(dihs[n][1:3])
                    print(dihs[n][1:3], np.rad2deg(dihedral_deviations[n]))

                
    print("\n\n")
    
    # Plot trajectory of ICs
    num_plots = int(log_bonds+log_bends+log_dihedrals)
    index = 0
    fig,ax = pt.subplots(num_plots,1,figsize=(10,30),sharex=True)
    if num_plots==1:
        ax = [ax]
    
    if log_bonds:
        for n in range(bond_values.shape[1]):
            if bond_deviations[n] > 1e-2:
                ax[index].plot(range(len(structures)),bond_values[:,n],marker='o',label=str(bonds[n]))
            elif bond_deviations_old[n] > 1e-2:
                ax[index].plot(range(len(structures)),bond_values[:,n],marker='o')
        ax[index].set_title('Bonds')
        ax[index].legend(bbox_to_anchor=(1.1,.5), loc='center left',frameon=False)
        index+=1
    

    if log_bends:
        for n in range(bend_values.shape[1]):
            if bend_deviations[n] > 2*deg:
                ax[index].plot(range(len(structures)),np.rad2deg(bend_values[:,n]),marker='o',label=str(bends[n]))
            elif bend_deviations_old[n] > 2*deg:
                ax[index].plot(range(len(structures)),np.rad2deg(bend_values[:,n]),linestyle='--',marker='o',alpha=0.5)
        ax[index].set_title('Bends')
        ax[index].legend(bbox_to_anchor=(1.1,.5), loc='center left',frameon=False)
        index+=1

    if log_dihedrals:
        for n in range(dih_values.shape[1]):
            if dihedral_deviations[n] > 10*deg:
                ax[index].plot(range(len(structures)),np.rad2deg(dih_values[:,n]),marker='o',label=str(dihs[n]))
            elif dihedral_deviations_old[n] > 10*deg:
                ax[index].plot(range(len(structures)),np.rad2deg(dih_values[:,n]),linestyle='--',marker='o',alpha=0.5)

        ax[index].legend(bbox_to_anchor=(1.1,.5), loc='center left',frameon=False)
        ax[index].set_title('Dihedrals')
        
    ax[index].set_xticks(range(len(structures)))
    ax[index].set_xticklabels(structures.keys())
    
    if save_png:
        path = pr.root_path +  pr.project_path.split('/')[0] + '/input_files/' + '/'.join(pr.project_path.split('/')[1:]) + '{}/ICs/'.format(ff.block_name)
        pathlib.Path(path).mkdir(parents=True, exist_ok=True) # make directory if it does not exist
        pt.savefig(path+'{}.png'.format(name))
    
    pt.show()

### Execution

#### PART 1

In [ ]:
pr_tr = Project('barriers/triazine')

In [ ]:
# Gather structures & force fields

# ffpars_nodih will be calculated by the write_ffpars_nodih function
# ffpars_polysix will be calculated by the write_ffpars_polysix function

ffs_triazine = {}

for block in glob.glob('./input_files/barriers/triazine/*/'):
    block_name = block.split('/')[-2]
    print(block_name)
    ffpars_fns = [fn for fn in glob.glob(block+'ff_pars/*.txt')]    
    fn_ai = block+block_name+'_freq.fchk'
    tmp = pr.create_job(pr.job_type.Yaff,'tmp',delete_existing_job=True)
    tmp.load_chk(block+block_name+'_freq.chk')
    structure = tmp.structure # ai structure
    tmp.load_chk(block+block_name+'.chk')
    ffs_triazine[block_name] = FFobject(block_name,structure,tmp.ffatypes,tmp.ffatype_ids,tmp.bonds,ffpars_fns,fn_ai)

In [ ]:
dihed_list = {}
with open('./input_files/barriers/triazine/diheds.txt','r') as f:
    lines = f.readlines()

for line in lines:
    split = line.split()
    dihed_list[split[0]] = tuple(int(i) for i in split[1].split(','))

In [ ]:
# Structure info and rotation grid
# Degeneracy will be replaced by numerical value in the write_ffpars_nodih function
sinfos_triazine = {
    k.replace('-','_') : Sinfo(pr_tr, ffs_triazine[k], v, range_max=180) for k,v in dihed_list.items() if k in ffs_triazine
}

In [ ]:
sinfos_triazine['Adamantane_Nitrile_Triazine'].range_max=60 # should have symmetry of 60 degrees (30 if mirrored)

In [ ]:
# Create groups - each key should be a group here
sinfo_triazine_groups = {}
groups = [k.replace('-','_') for k in list(ffs_triazine.keys())]

for name in groups:
    group_info = {}
    for sname,sinfo in sinfos_triazine.items():
        if sname.startswith(name):
            group_info[sname] = sinfo
            
    if len(group_info)>0:     
        sinfo_triazine_groups[name] = group_info

for n,(k,v) in enumerate(sinfo_triazine_groups.items()):
    print(n,k,list(v.keys()))

In [ ]:
# If the rotation is significantly hindered, or the 
gic_triazine = {
        "Adamantane_Nitrile_Triazine" : "dihedral(freeze)=D(9,5,23,26)", # indices start from 1 instead of 0 for gic
        "T_brick_Nitrile_Triazine" : "FreezeAll\nAng0 = A(21,43,44)", # indices start from 1 instead of 0 for gic
      }

In [ ]:
# First check whether only the dihedrals are changing
for name,sinfo in sinfos_triazine.items():
    plot_IC_variations(name,sinfo)

In [ ]:
# Then alter the ff objects with the altered ffpars
for name,sinfo in sinfos_triazine.items():
    write_ffpars_nodih(name,sinfo)

In [ ]:
# Perform the scans
for name,sinfo in sinfos_triazine.items():
    if not name in gic_triazine:
        print(name)
        pr_sub = Project(sinfo.get_sub_project_name(name))
        grid = np.arange(sinfo.range_min, sinfo.range_max+sinfo.range_step, sinfo.range_step)
        for n,val in enumerate(grid):
            adapted_structure = sinfo.adapt_structure(sinfo.ff.structure,val)
            g16_job(pr_sub,adapted_structure,'job_{}'.format(n),run_time=2*60*60)

In [ ]:
# Perform the gic scans
for name,sinfo in sinfos_triazine.items():
    if name in gic_triazine:
        if name not in ['T_brick_Nitrile_Triazine']:
            print(name)
            pr_sub = Project(sinfo.get_sub_project_name(name))
            grid = np.arange(sinfo.range_min, sinfo.range_max+sinfo.range_step, sinfo.range_step)
            for n,val in enumerate(grid):
                adapted_structure = sinfo.adapt_structure(sinfo.ff.structure,val)
                g16_gic_job(pr_sub,adapted_structure,'job_{}'.format(n),suffix=gic_triazine[name],run_time=10*60*60)

#### PART 2

In [ ]:
pr_ext_0 = Project('barriers/ctfs')
pr_ext_1 = Project('barriers/ctf-1')

In [ ]:
# Gather structures & force fields

# ffpars_nodih will be calculated by the write_ffpars_nodih function
# ffpars_polysix will be calculated by the write_ffpars_polysix function

ffs_ext = {}

for block in glob.glob('./input_files/barriers/Sara/*/'):
    block_name = block.split('/')[-2]
    if block_name in ['Phenyl_Nitrile_Triazine','F_Nitrile_Triazine','BiPhenyl_Nitrile_Triazine']:
        print(block_name)
        ffpars_fns = [fn for fn in glob.glob(block+'ff_pars/*.txt') if not 'uff' in fn] 
        fn_ai = block+block_name+'_freq.fchk'
        tmp = pr.create_job(pr.job_type.Yaff,'tmp',delete_existing_job=True)
        tmp.load_chk(block+block_name+'_freq.chk')
        structure = tmp.structure # ai structure
        tmp.load_chk(block+'system_opt.chk')
        ffs_ext[block_name] = FFobject(block_name,structure,tmp.ffatypes,tmp.ffatype_ids,tmp.bonds,ffpars_fns,fn_ai)
        
        
        
    
for block in glob.glob('./input_files/barriers/ctf-1/*/'):
    block_name = block.split('/')[-2]
    if block_name in ['Triazine']:
        ei = 'mbis_gaussian'
        vdw = 'mm3'

        name = block_name+'_{}_{}'.format(ei,vdw)
        print(name)
        ffpars_fns = [
                        block+'cov/pars_cluster_cov_{}_{}.txt'.format(ei,vdw),
                        block+'ei/pars_cluster_{}.txt'.format(ei),
                        block+'vdw/pars_{}.txt'.format(vdw)
                     ]

        fn_ai = block+block_name+'_freq.fchk'
        tmp = pr.create_job(pr.job_type.Yaff,'tmp',delete_existing_job=True)
        tmp.load_chk(block+block_name+'_freq.chk')
        structure = tmp.structure # ai structure
        tmp.load_chk(block+block_name+'_ai.chk')
        ffs_ext[name] = FFobject(block_name,structure,tmp.ffatypes,tmp.ffatype_ids,tmp.bonds,ffpars_fns,fn_ai)

In [ ]:
# Structure info and rotation grid
# Degeneracy will be replaced by numerical value in the write_ffpars_nodih function
sinfos_ext = {
    'BiPhenyl_Nitrile_Triazine_triazine' : Sinfo(pr_ext_0,ffs_ext['BiPhenyl_Nitrile_Triazine'],(21,20,0,5)), # phenyl wrt triazine rot barrier
    'F_Nitrile_Triazine'                 : Sinfo(pr_ext_0,ffs_ext['F_Nitrile_Triazine'],(11,10,4,2)), # phenyl with all F functionality wrt triazine rot barrier
    'Phenyl_Nitrile_Triazine'            : Sinfo(pr_ext_0,ffs_ext['Phenyl_Nitrile_Triazine'],(11,10,0,5)), # phenyl wrt triazine barrier    
    
    'Triazine_mbis_gaussian_mm3'         : Sinfo(pr_ext_1,ffs_ext['Triazine_mbis_gaussian_mm3'],(1,0,6,7)), # triazine rot barrier
}

In [ ]:
# Create groups - each key should be a group here
sinfo_ext_groups = {}
groups = [k.replace('-','_') for k in list(ffs_ext.keys())]

for name in groups:
    group_info = {}
    for sname,sinfo in sinfos_ext.items():
        if sname.startswith(name):
            group_info[sname] = sinfo
            
    if len(group_info)>0:     
        sinfo_ext_groups[name] = group_info

for n,(k,v) in enumerate(sinfo_ext_groups.items()):
    print(n,k,list(v.keys()))

In [ ]:
# First check whether only the dihedrals are changing
for name,sinfo in sinfos_ext.items():
    plot_IC_variations(name,sinfo)

In [ ]:
# Then alter the ff objects with the altered ffpars
for name,sinfo in sinfos_ext.items():
    write_ffpars_nodih(name,sinfo)

In [ ]:
# Perform the scans
for name,sinfo in sinfos_ext.items():
    print(name)
    pr_sub = Project(sinfo.get_sub_project_name(name))
    grid = np.arange(sinfo.range_min, sinfo.range_max+sinfo.range_step, sinfo.range_step)
    for n,val in enumerate(grid):
        adapted_structure = sinfo.adapt_structure(sinfo.ff.structure,val)
        g16_job(pr_sub,adapted_structure,'job_{}'.format(n),run_time=2*60*60)

## AI barrier data parsing

### Functions

In [ ]:
# Gather AI data for each project with pyiron table

def cluster_pos(positions, rvecs, indices):
    import itertools
    pos = copy.copy(positions)
    a,b,c = rvecs[0], rvecs[1], rvecs[2]
    for n in indices[1:]:
        images = pos[n] + np.array([sum(n * vec for n, vec in zip(ns, [a,b,c])) for ns in itertools.product([-1,0,1],repeat=3)]) # 3 dimensions
        distances = np.linalg.norm(images-pos[indices[0]], axis=-1)
        pos[n] = images[np.argmin(distances)]
    return pos

def normal(v1,v2):
    n = np.cross(v1,v2, axis=-1)
    return (n.T/np.linalg.norm(n,axis=-1)).T

def get_dihedral(job,indices,scan=False,cluster=True):
    pos = job['output/generic/positions']
    
    # If an optimization happened only consider final frame, if this was a scan job, get all data
    if not scan and pos.shape[0]>1:
        pos = pos[-1,:,:].reshape(1,-1,3)

    # pbc might have moved atoms
    if cluster and job.get('output/generic/cells') is not None:
        cells = job.get('output/generic/cells')
        if len(cells.shape)==3:
            cells = cells[-1,:,:]
        pos = cluster_pos(pos, cells, indices)

    i1,i2,i3,i4 = indices
    n1 = normal(pos[:,i2,:] - pos[:,i1,:], pos[:,i3,:] - pos[:,i2,:])
    n2 = normal(pos[:,i3,:] - pos[:,i2,:], pos[:,i4,:] - pos[:,i3,:])
    try:
        phi = np.arccos(np.einsum('ij,ij->i',n1,n2))
    except FloatingPointError:
        phi = np.arccos(np.clip(np.einsum('ij,ij->i',n1,n2),-1.,1.))
    signs = np.sign(np.einsum('ij,ij->i',n1,pos[:,i4,:] - pos[:,i3,:]))
    phi *= signs
    phi[np.isclose(phi,-np.pi,atol=1e-4)] = -phi[np.isclose(phi,-np.pi,atol=1e-4)]
    
    return phi

def get_e(job):
    energy = job['output/generic/energy_tot']*electronvolt
    if isinstance(energy,np.ndarray):
        return energy[-1]
    else:
        return energy

def get_table(pr,name,grid,sinfo):
    # Create table
    name = name.translate({ord(c): None for c in '!@#$(),/-'})
    table = pr.create_table('table_'+name, delete_existing_job=True)
    
    def db_filter(job_table):
        return (job_table.status == "finished") & (job_table.hamilton != 'TableJob') & (job_table.job.str.contains('job_', regex=False))
    
    #def get_angle(job,indices=sinfo.indices):
    #    return grid[int(job.job_name.split('_')[1])] # in degrees
    
    table.db_filter_function = db_filter
    #table.convert_to_object = True  
    
    # Introduce property function
    if sinfo.fit_indices is not None:
        indices = sinfo.fit_indices
        ref_indices = sinfo.indices
        table.add["reference_angle"] = lambda job: get_dihedral(job,ref_indices)[-1]/deg
    else:
        indices = sinfo.indices
        
    table.add["angle"] = lambda job: get_dihedral(job,indices,cluster=False)[-1]/deg   # gaussian can never have pbcs
    table.add["E"] = get_e
     
    # Run it
    table.run()
    
    # Get the dataframe
    return table.get_dataframe()

# Plotting function
def plot(name,aid):
    print(name)
    fig,ax = pt.subplots(1,2,sharey=True,figsize=(25,5))
    
    angles = aid.sort_values(by=['angle']).angle
    energy = aid.sort_values(by=['angle']).E/kjmol

    ax[0].plot(angles,energy, 'bo--')
    ax[0].set_xlabel(u'Dihedral angle (°)')
    ax[0].set_ylabel('E (kJ/mol)')
    ax[0].set_xlim([min(angles), max(angles)])


    xc = np.array(np.cos(np.deg2rad(angles)))
    yc = energy

    ax[1].plot(xc,yc,'bo--')
    ax[1].set_xlabel(u'cos $\phi$')
    #ax[1].set_ylabel('E (kJ/mol)')
    ax[1].set_xlim([min(xc), max(xc)])
    ax[1].axvline(0,0,100,c='k')
    pt.show()

### Execution

#### PART 1

In [ ]:
# Gather data
ai_data_triazine = {}
for name,sinfo in sinfos_triazine.items():
    print(sinfo.get_sub_project_name(name))
    pr_sub = Project(sinfo.get_sub_project_name(name))
    grid = np.arange(sinfo.range_min, sinfo.range_max+sinfo.range_step, sinfo.range_step)
    ai_data_triazine[name] = get_table(pr_sub,name,grid,sinfo)
    
    # These structures were not perfectly generated
    ai_data_triazine[name].angle = grid


In [ ]:
# Plotting
for name,aid in ai_data_triazine.items():
    plot(name,aid)

#### PART 2

In [ ]:
# Gather data
ai_data_ext = {}
for name,sinfo in sinfos_ext.items():
    print(sinfo.get_sub_project_name(name))
    
    if name in ['Triazine_mbis_gaussian_mm3']:
        pr_sub = Project(sinfo.get_sub_project_name('Triazine_hi_mm3'))
    else:
        pr_sub = Project(sinfo.get_sub_project_name(name))
    grid = np.arange(sinfo.range_min, sinfo.range_max+sinfo.range_step, sinfo.range_step)
    ai_data_ext[name] = get_table(pr_sub,name,grid,sinfo)

In [ ]:
# Plotting
for name,aid in ai_data_ext.items():
    plot(name,aid)

## FF barrier data calculation

### Execution

#### PART 1

In [ ]:
# Perform the scans
for name,sinfo in sinfos_triazine.items():
    if name not in gic_triazine:
        yaff_scan_job(sinfo.pr, 'scan_'+name+'_nodih', sinfo, sinfo.ff.ffpars_nodih)
        yaff_scan_job(sinfo.pr, 'scan_'+name+'_wdih', sinfo, sinfo.ff.ffpars)

In [ ]:
# Perform the scans
for name,sinfo in sinfos_triazine.items():
    if name in gic_triazine and name in ['Adamantane_Nitrile_Triazine']:
        pr_sub_g16 = Project(sinfo.get_sub_project_name(name))
        structures = [pr_sub_g16.load('job_{}'.format(n)).get_structure() for n,_ in enumerate(np.arange(sinfo.range_min, sinfo.range_max+sinfo.range_step, sinfo.range_step))]
        yaff_scan_structures_job(sinfo.pr, 'scan_'+name+'_nodih', sinfo, sinfo.ff.ffpars_nodih, structures)
        yaff_scan_structures_job(sinfo.pr, 'scan_'+name+'_wdih', sinfo, sinfo.ff.ffpars, structures)

#### PART 2

In [ ]:
# Perform the scans - no GIC
for name,sinfo in sinfos_ext.items():
    name = name.translate({ord(c): None for c in '!@#$(),/-'})
    yaff_scan_job(sinfo.pr, 'scan_'+name+'_nodih', sinfo, sinfo.ff.ffpars_nodih)
    yaff_scan_job(sinfo.pr, 'scan_'+name+'_wdih', sinfo, sinfo.ff.ffpars)

## FF barrier data parsing

### Functions

In [1]:
@dataclass
class FF_data:
    table_nodih: object
    table_wdih: object
    table_polysix: object

NameError: name 'dataclass' is not defined

In [ ]:
import pandas as pd

def get_tables_ff(name,sinfo,grid):
    name = name.translate({ord(c): None for c in '!@#$(),/-'})
    nodih = sinfo.pr.load('scan_'+name+'_nodih')
    wdih = sinfo.pr.load('scan_'+name+'_wdih')
    
    if nodih is not None:
        epot_contrib_names = [contrib_name.decode("utf-8") for contrib_name in nodih['output/generic/epot_contrib_names']]
    else:
        epot_contrib_names = [contrib_name.decode("utf-8") for contrib_name in wdih['output/generic/epot_contrib_names']]
    
    if sinfo.fit_indices is not None:
        angles = get_dihedral(nodih,sinfo.fit_indices,scan=True,cluster=False)/deg
    else:
        angles = get_dihedral(nodih,sinfo.indices,scan=True,cluster=False)/deg
        
    df_nodih = pd.DataFrame(np.array([angles,*nodih['output/generic/epot_contribs'].T*electronvolt,nodih['output/generic/energy_pot']*electronvolt]).T,
                   columns=['angle', *epot_contrib_names, 'e_tot'])
    df_wdih = pd.DataFrame(np.array([angles,*wdih['output/generic/epot_contribs'].T*electronvolt,wdih['output/generic/energy_pot']*electronvolt]).T,
                   columns=['angle', *epot_contrib_names, 'e_tot'])

    # Get the dataframe
    return df_nodih, df_wdih

def get_table_polysix(name,sinfo,grid):
    name = name.translate({ord(c): None for c in '!@#$(),/-'})
    polysix = sinfo.pr.load('scan_'+name+'_polysix')
    if sinfo.fit_indices is not None:
        angles = get_dihedral(polysix,sinfo.fit_indices,scan=True,cluster=False)/deg
    else:
        angles = get_dihedral(polysix,sinfo.indices,scan=True,cluster=False)/deg
    return pd.DataFrame(np.array([angles,polysix['output/generic/energy_pot']*electronvolt]).T,
                   columns=['angle', 'e_tot'])

def centre(arr):
    return (arr-np.nanmin(arr))

# Plotting and fitting function
def fit_and_plot_ff(name,sinfo,aid,ffd,save_png=False):
    from scipy import optimize

    print(name)
    odd = True # will select correct poly function
    
    vdw_names = ['pair_mm3','pair_lj','pair_mm3cap']
    if len(ffd.table_nodih)>0:
        vdw_name = [name for name in vdw_names if name in list(ffd.table_nodih.columns)]
    else:
        vdw_name = [name for name in vdw_names if name in list(ffd.table_wdih.columns)]
    assert len(vdw_name)==1
    vdw_name = vdw_name[0]
    
    
    # FF data
    if len(ffd.table_nodih)>0:
        angles       = ffd.table_nodih.sort_values(by=['angle']).angle
        FF_val_nodih = ffd.table_nodih.sort_values(by=['angle']).valence/kjmol
        FF_ei_nodih  = ffd.table_nodih.sort_values(by=['angle']).pair_ei/kjmol
        FF_vdw_nodih = ffd.table_nodih.sort_values(by=['angle'])[vdw_name]/kjmol
        FF_tot_nodih = ffd.table_nodih.sort_values(by=['angle']).e_tot/kjmol
        
    else:
        angles       = ffd.table_wdih.sort_values(by=['angle']).angle
    
    FF_val_wdih = ffd.table_wdih.sort_values(by=['angle']).valence/kjmol
    FF_ei_wdih  = ffd.table_wdih.sort_values(by=['angle']).pair_ei/kjmol
    FF_vdw_wdih = ffd.table_wdih.sort_values(by=['angle'])[vdw_name]/kjmol
    FF_tot_wdih = ffd.table_wdih.sort_values(by=['angle']).e_tot/kjmol
    
    # AI data
    if aid is not None:
        ai_angles   = aid.sort_values(by=['angle']).angle
        ai_energy   = aid.sort_values(by=['angle']).E/kjmol
        try:
            assert np.all(np.isclose(np.asarray(ai_angles),np.asarray(angles),atol=1e-3))
        except AssertionError:
            print(np.asarray(ai_angles),np.asarray(angles))
            print(np.isclose(np.asarray(ai_angles),np.asarray(angles),atol=1e-4))
            raise AssertionError
    
    
    # If angle range is limited to 90 degrees mirror the behaviour to 180
    def mirror(array, angle=False):
        if angle:
            return np.hstack((array, 180-array[:-1][::-1]))
        else:
            return np.hstack((array, array[:-1][::-1]))
    
    
    def mirror60(array, angle=False):
        if angle:
            return np.hstack((array, 120-array[:-1][::-1], 120+array[1:]))
        else:
            return np.hstack((array, array[:-1][::-1], array[1:]))
        
        
    if np.isclose(np.max(angles),90,atol=1e-5):
        print('Mirroring - 90')
        odd = False
        angles       = mirror(angles,angle=True)
        if len(ffd.table_nodih)>0:
            FF_val_nodih = mirror(FF_val_nodih)
            FF_ei_nodih  = mirror(FF_ei_nodih)
            FF_vdw_nodih = mirror(FF_vdw_nodih)
            FF_tot_nodih = mirror(FF_tot_nodih)
        
        FF_val_wdih = mirror(FF_val_wdih)
        FF_ei_wdih  = mirror(FF_ei_wdih)
        FF_vdw_wdih = mirror(FF_vdw_wdih)
        FF_tot_wdih = mirror(FF_tot_wdih)
        
        if aid is not None:
            ai_energy   = mirror(ai_energy)
    elif np.isclose(np.max(angles),60,atol=1e-5):
        print('Mirroring - 60')
        angles       = mirror60(angles,angle=True)
        if len(ffd.table_nodih)>0:
            FF_val_nodih = mirror60(FF_val_nodih)
            FF_ei_nodih  = mirror60(FF_ei_nodih)
            FF_vdw_nodih = mirror60(FF_vdw_nodih)
            FF_tot_nodih = mirror60(FF_tot_nodih)
        
        FF_val_wdih = mirror60(FF_val_wdih)
        FF_ei_wdih  = mirror60(FF_ei_wdih)
        FF_vdw_wdih = mirror60(FF_vdw_wdih)
        FF_tot_wdih = mirror60(FF_tot_wdih)
        
        if aid is not None:
            ai_energy   = mirror60(ai_energy)
    else:
        angles      = np.asarray(angles)
        
        if len(ffd.table_nodih)>0:
            FF_val_nodih = np.asarray(FF_val_nodih)
            FF_ei_nodih  = np.asarray(FF_ei_nodih)
            FF_vdw_nodih = np.asarray(FF_vdw_nodih)
            FF_tot_nodih = np.asarray(FF_tot_nodih)
        
        FF_val_wdih = np.asarray(FF_val_wdih)
        FF_ei_wdih  = np.asarray(FF_ei_wdih)
        FF_vdw_wdih = np.asarray(FF_vdw_wdih)
        FF_tot_wdih = np.asarray(FF_tot_wdih)
        
        if aid is not None:
            ai_energy   = np.asarray(ai_energy)
        
    def get_poly(odd):
        # Fit functions
        def poly(x,b,d,f,g):
            return g + b*x**2 + d*x**4 + f*x**6

        def poly_odd(x,a,b,c,d,e,f,g):
            return g + a*x + b*x**2 + c*x**3 + d*x**4 + e*x**5 + f*x**6
        
        if odd:
            return poly_odd
        else:
            return poly
        
    fpoly = get_poly(odd)
    
    if aid is not None:
        # Calculate difference and fit a polysix term to the difference, focussing on the minimum
        diff = centre(ai_energy - FF_tot_nodih)

        cosr = np.cos(np.deg2rad(angles)) # range as cos(x)
        fitr = cosr[~np.isnan(diff)]
        fitf = diff[~np.isnan(diff)]

        sigma=np.ones(len(fitf))
        #sigma[np.argmin(fitf)] = 1e-4 # force minimum position
        #sigma[centre(ai_energy)<20*kjmol] = 1e-4
        p,pcov = optimize.curve_fit(fpoly, fitr, fitf, sigma=sigma)
        print(p)


        # Plot FF and AI data and compare with fit
        fig,ax = pt.subplots(2,2,sharex=True,sharey=True,figsize=(25,10))
        ax[0,0].plot(angles,centre(FF_val_nodih), label='valence')
        ax[0,0].plot(angles,centre(FF_ei_nodih), label='ei')
        ax[0,0].plot(angles,centre(FF_vdw_nodih), label='vdw')
        ax[0,0].plot(angles,centre(FF_tot_nodih), ls='--', label='tot')
        ax[0,0].plot(angles,centre(ai_energy), label='AI')
        ax[0,0].set_xlabel(r'$\psi$ ($^\circ$)')
        ax[0,0].set_ylabel('E (kJ/mol)')
        ax[0,0].legend(bbox_to_anchor=(1.0,.5), loc=6)
        ax[0,0].set_title('Energy barrier wo dihedral term')

        ax[0,1].plot(angles,centre(FF_val_wdih), label='valence')
        ax[0,1].plot(angles,centre(FF_ei_wdih), label='ei')
        ax[0,1].plot(angles,centre(FF_vdw_wdih), label='vdw')
        ax[0,1].plot(angles,centre(FF_tot_wdih), ls='--', label='tot')
        ax[0,1].plot(angles,centre(ai_energy), label='AI')
        ax[0,1].set_xlabel(r'$\psi$ ($^\circ$)')
        ax[0,1].set_ylabel('E (kJ/mol)')
        ax[0,1].legend(bbox_to_anchor=(1.0,.5), loc=6)
        ax[0,1].set_title('Energy barrier w dihedral term')

        ax[1,0].plot(angles,diff, c='r', label='diff')
        ax[1,0].plot(angles,fpoly(cosr,*p), 'k--', label='fit')
        ax[1,0].set_xlabel(r'$\psi$ ($^\circ$)')
        ax[1,0].set_ylabel('E (kJ/mol)')
        ax[1,0].legend(bbox_to_anchor=(1.0,.5), loc=6)
        ax[1,0].set_title('Fit (wrt diff)')

        ax[1,1].plot(angles,centre(ai_energy), label='AI', c='r')
        ax[1,1].plot(angles,centre(fpoly(cosr,*p) + FF_tot_nodih), 'k--',label='fit')
        ax[1,1].set_xlabel(r'$\psi$ ($^\circ$)')
        ax[1,1].set_ylabel('E (kJ/mol)')
        ax[1,1].legend(bbox_to_anchor=(1.0,.5), loc=6)
        ax[1,1].set_title('Fit (wrt AI barrier)')
        
        '''
        ax[2,0].plot(cosr,diff, c='r', label='diff')
        ax[2,0].plot(cosr,fpoly(cosr,*p), 'k--', label='fit')
        ax[2,0].set_xlabel(r'$\cos \psi$')
        ax[2,0].set_ylabel('E (kJ/mol)')
        ax[2,0].legend(bbox_to_anchor=(1.0,.5), loc=6)
        ax[2,0].set_title('Fit (wrt diff)')

        ax[2,1].plot(cosr,centre(ai_energy), label='AI', c='r')
        ax[2,1].plot(cosr,centre(fpoly(cosr,*p) + FF_tot_nodih), 'k--',label='fit')
        ax[2,1].set_xlabel(r'$\cos \psi$ ($^\circ$)')
        ax[2,1].set_ylabel('E (kJ/mol)')
        ax[2,1].legend(bbox_to_anchor=(1.0,.5), loc=6)
        ax[2,1].set_title('Fit (wrt AI barrier)')
        '''
        
    else:
        # Plot FF and AI data and compare with fit
        if len(ffd.table_nodih)>0:
            fig,ax = pt.subplots(1,2,sharex=True,sharey=True,figsize=(30,10))
            ax[0].plot(angles,centre(FF_val_nodih), label='valence')
            ax[0].plot(angles,centre(FF_ei_nodih), label='ei')
            ax[0].plot(angles,centre(FF_vdw_nodih), label='vdw')
            ax[0].plot(angles,centre(FF_tot_nodih), ls='--', label='tot')
            ax[0].set_xlabel(r'$\psi$ ($^\circ$)')
            ax[0].set_ylabel('E (kJ/mol)')
            ax[0].legend(bbox_to_anchor=(1.0,.5), loc=6)
            ax[0].set_title('Energy barrier wo dihedral term')

            ax[1].plot(angles,centre(FF_val_wdih), label='valence')
            ax[1].plot(angles,centre(FF_ei_wdih), label='ei')
            ax[1].plot(angles,centre(FF_vdw_wdih), label='vdw')
            ax[1].plot(angles,centre(FF_tot_wdih), ls='--', label='tot')
            ax[1].set_xlabel(r'$\psi$ ($^\circ$)')
            ax[1].set_ylabel('E (kJ/mol)')
            ax[1].legend(bbox_to_anchor=(1.0,.5), loc=6)
            ax[1].set_title('Energy barrier w dihedral term')
        else:
            fig,ax = pt.subplots(1,1,sharex=True,sharey=True,figsize=(30,10))
            ax.plot(angles,centre(FF_val_wdih), label='valence')
            ax.plot(angles,centre(FF_ei_wdih), label='ei')
            ax.plot(angles,centre(FF_vdw_wdih), label='vdw')
            ax.plot(angles,centre(FF_tot_wdih), ls='--', label='tot')
            ax.set_xlabel(r'$\psi$ ($^\circ$)')
            ax.set_ylabel('E (kJ/mol)')
            ax.legend(bbox_to_anchor=(1.0,.5), loc=6)
            ax.set_title('Energy barrier w dihedral term')

    
    if save_png:
        path = sinfo.pr.root_path +  sinfo.pr.project_path.split('/')[0] + '/input_files/' + '/'.join(sinfo.pr.project_path.split('/')[1:]) + sinfo.ff.block_name + '/fits/'
        pathlib.Path(path).mkdir(parents=True, exist_ok=True) # make directory if it does not exist
        pt.savefig(path+'{}.pdf'.format(name),bbox_inches='tight')
    
    pt.show()
    
    if aid is not None:
        return p

### Execution

#### PART 1

In [ ]:
# Gather data
ff_triazine_datas = {}

# table_polysix will be calculated in 6: Testing polysix terms 
for name,sinfo in sinfos_triazine.items():
    grid = np.arange(sinfo.range_min, sinfo.range_max+sinfo.range_step, sinfo.range_step)
    ff_triazine_datas[name] = FF_data(*get_tables_ff(name,sinfo,grid),None)

In [ ]:
# Fitting and Plotting
ps_triazine = {}
for name,ffd in ff_triazine_datas.items():
    ps_triazine[name] = fit_and_plot_ff(name,sinfos_triazine[name],ai_data_triazine[name],ffd,save_png=True)

#### PART 2

In [ ]:
# Gather data
ff_ext_datas = {}

# table_polysix will be calculated in 6: Testing polysix terms 
for name,sinfo in sinfos_ext.items():
    print(name)
    grid = np.arange(sinfo.range_min, sinfo.range_max+sinfo.range_step, sinfo.range_step)
    ff_ext_datas[name] = FF_data(*get_tables_ff(name,sinfo,grid),None)

In [ ]:
# Fitting and Plotting
ps_ext = {}
for name,ffd in ff_ext_datas.items():
    ps_ext[name] = fit_and_plot_ff(name,sinfos_ext[name],ai_data_ext[name],ffd,save_png=True)

## Generating polysix parameterfile

### Functions

In [ ]:
import pathlib

def write_ffpars_polysix(sinfo,fit_params,name=None,subname=None,strict_indices=False,ref_val=0,override_degeneracy=None):    
    # Make yaff system and adapt it to angle 0
    adapted_structure = sinfo.adapt_structure(sinfo.ff.structure,ref_val)
    yaff_system = System(sinfo.ff.structure.numbers, adapted_structure.positions, ffatypes=sinfo.ff.ffatypes, ffatype_ids=sinfo.ff.ffatype_ids, bonds=sinfo.ff.bonds)
    
    # Log all dihedrals
    dihs = []
    dihs_ids = []
    dihs_cos = []
    for dih in yaff_system.iter_dihedrals():
        dihs.append(dih)
        dihs_ids.append([yaff_system.ffatypes[yaff_system.ffatype_ids[i]] for i in dih])
        dihs_cos.append(DihedCos(*dih))

    # Filter out the appropriate ones
    if sinfo.fit_indices is not None:
        dihedral_idx = sinfo.fit_indices
    else:
        dihedral_idx = sinfo.indices
    cv_dih = []
    cv_dihcos = []
    for n,dih in enumerate(dihs):
        if (dih[1]==dihedral_idx[1] and dih[2]==dihedral_idx[2]) or (dih[1]==dihedral_idx[2] and dih[2]==dihedral_idx[1]):
            if not strict_indices or (strict_indices and (dih == dihedral_idx or dih[::-1] == dihedral_idx)):
                if dihs_ids[n] not in cv_dih and dihs_ids[n][::-1] not in cv_dih:
                    cv_dih.append(dihs_ids[n])
                    cv_dihcos.append(dihs_cos[n])
    
    # Evaluate these dihedrals
    ICl = InternalCoordinateList(DeltaList(yaff_system))
    for dih in cv_dihcos:
        ICl.add_ic(dih)
    ICl.dlist.forward()
    ICl.forward()
    

    values = [int(np.round(c.get_last_computed_value())) for c in cv_dihcos]

    powers = [[c**m for m in [1,2,3,4,5,6]] for c in values] # to get the sign right of odd and even powers
    w = max([len(n) for dih in cv_dih for n in dih]) # to get nice alligned parameter files
    
    # Evaluate fit params to 6
    fit_params = fit_params[:-1] # do not account for constant term
    if not len(fit_params)==6:
        fit_params=[0,fit_params[0],0,fit_params[1],0,fit_params[2]]
        
    ffpars_polysix = ''
    ffpars_polysix += '''# TORSCPOLYSIX
#---------
TORSCPOLYSIX:UNIT  C1 kjmol
TORSCPOLYSIX:UNIT  C2 kjmol
TORSCPOLYSIX:UNIT  C3 kjmol
TORSCPOLYSIX:UNIT  C4 kjmol
TORSCPOLYSIX:UNIT  C5 kjmol
TORSCPOLYSIX:UNIT  C6 kjmol

'''
    for n,dih in enumerate(cv_dih):
        if override_degeneracy is not None:
            tmp = np.array(fit_params)/override_degeneracy
        else:
            tmp = np.array(fit_params)/sinfo.degeneracy
        power = powers[n]
        v = [power[m]*tmp[m] for m in range(len(power))]
        ffpars_polysix += 'TORSCPOLYSIX:PARS  '
        ffpars_polysix += "{:>{w}}  {:>{w}}  {:>{w}}  {:>{w}}  {: 1.10e}  {: 1.10e}  {: 1.10e}  {: 1.10e}  {: 1.10e}  {: 1.10e}\n".format(dih[0],dih[1],dih[2],dih[3],v[0],v[1],v[2],v[3],v[4],v[5],w=w)
    
    
    
    path = sinfo.pr.root_path +  sinfo.pr.project_path.split('/')[0] + '/input_files/' + '/'.join(sinfo.pr.project_path.split('/')[1:]) + sinfo.ff.block_name + '/polysix/' 
    path = path + '{}/'.format(name) if name is not None else path
    pathlib.Path(path).mkdir(parents=True, exist_ok=True) # make directory if it does not exist
    
    polysix_path = path+'pars_polysix.txt' if subname is None else path+'pars_polysix_{}.txt'.format(subname)
    
    with open(polysix_path, 'w') as f:
        f.write(ffpars_polysix)
    
    # Adapt polysix parameters in sinfo
    sinfo.ff.ffpars_polysix = ffpars_polysix


### Execution

#### PART 1

In [ ]:
for key,val in sinfo_triazine_groups.items():
    # Group polysix parameter files per group
    print('{}: {}'.format(key,','.join(list(val.keys()))))
    names = [k for k in val.keys()]
    sinfos = [val[k] for k in val.keys()]
    for n,sinfo in enumerate(sinfos):
        write_ffpars_polysix(sinfo,ps_triazine[names[n]],name='_'.join(key.split('_')[1:]),subname=None) # we don't need subname here, only 1 polysix parameter file per key

In [ ]:
for key,val in sinfo_ext_groups.items():
    # Group polysix parameter files per group
    print('{}: {}'.format(key,','.join(list(val.keys()))))
    names = [k for k in val.keys()]
    sinfos = [val[k] for k in val.keys()]
    for n,sinfo in enumerate(sinfos):
        write_ffpars_polysix(sinfo,ps_ext[names[n]], name=None, subname=names[n].split('-')[-1])

## Testing polysix terms

### Functions

In [ ]:
# Plotting and fitting function
def plot_ff_polysix(name,aid,ffd):
    print(name)
    
    vdw_names = ['pair_mm3','pair_lj','pair_mm3cap']
    vdw_name = [name for name in vdw_names if name in list(ffd.table_nodih.columns)]
    assert len(vdw_name)==1
    vdw_name = vdw_name[0]
    
    # FF data
    angles       = ffd.table_nodih.sort_values(by=['angle']).angle
    FF_val_nodih = ffd.table_nodih.sort_values(by=['angle']).valence/kjmol
    FF_ei_nodih  = ffd.table_nodih.sort_values(by=['angle']).pair_ei/kjmol
    FF_vdw_nodih = ffd.table_nodih.sort_values(by=['angle'])[vdw_name]/kjmol
    FF_tot_nodih = ffd.table_nodih.sort_values(by=['angle']).e_tot/kjmol
    
    FF_val_wdih = ffd.table_wdih.sort_values(by=['angle']).valence/kjmol
    FF_ei_wdih  = ffd.table_wdih.sort_values(by=['angle']).pair_ei/kjmol
    FF_vdw_wdih = ffd.table_wdih.sort_values(by=['angle'])[vdw_name]/kjmol
    FF_tot_wdih = ffd.table_wdih.sort_values(by=['angle']).e_tot/kjmol
    
    poly_angles  = ffd.table_polysix.sort_values(by=['angle']).angle
    FF_poly      = ffd.table_polysix.sort_values(by=['angle']).e_tot/kjmol
    
    # AI data
    ai_angles   = aid.sort_values(by=['angle']).angle
    ai_energy   = aid.sort_values(by=['angle']).E/kjmol
    
    try:
        assert np.all(np.isclose(np.asarray(ai_angles),np.asarray(angles)))
        assert np.all(np.isclose(np.asarray(poly_angles),np.asarray(angles), atol=1e-4))
    except AssertionError:
        print(np.asarray(ai_angles)-np.asarray(angles))
        print(np.asarray(poly_angles)-np.asarray(angles))
        raise AssertionError

    
    # If angle range is limited to 90 degrees mirror the behaviour to 180
    def mirror(array, angle=False):
        if angle:
            return np.hstack((array, 180-array[:-1][::-1]))
        else:
            return np.hstack((array, array[:-1][::-1]))
        
    def mirror60(array, angle=False):
        if angle:
            return np.hstack((array, 120-array[:-1][::-1], 120+array[1:]))
        else:
            return np.hstack((array, array[:-1][::-1], array[1:]))
        
    if np.max(angles)==90:
        print('Mirroring')
        angles       = mirror(angles,angle=True)
        FF_val_nodih = mirror(FF_val_nodih)
        FF_ei_nodih  = mirror(FF_ei_nodih)
        FF_vdw_nodih = mirror(FF_vdw_nodih)
        FF_tot_nodih = mirror(FF_tot_nodih)
        
        FF_val_wdih = mirror(FF_val_wdih)
        FF_ei_wdih  = mirror(FF_ei_wdih)
        FF_vdw_wdih = mirror(FF_vdw_wdih)
        FF_tot_wdih = mirror(FF_tot_wdih)
        
        FF_poly   = mirror(FF_poly)
        ai_energy = mirror(ai_energy)
        
    elif np.max(angles)==60:
        print('Mirroring')
        angles       = mirror60(angles,angle=True)
        FF_val_nodih = mirror60(FF_val_nodih)
        FF_ei_nodih  = mirror60(FF_ei_nodih)
        FF_vdw_nodih = mirror60(FF_vdw_nodih)
        FF_tot_nodih = mirror60(FF_tot_nodih)
        
        FF_val_wdih = mirror60(FF_val_wdih)
        FF_ei_wdih  = mirror60(FF_ei_wdih)
        FF_vdw_wdih = mirror60(FF_vdw_wdih)
        FF_tot_wdih = mirror60(FF_tot_wdih)
        
        FF_poly   = mirror60(FF_poly)
        ai_energy = mirror60(ai_energy)
    else:
        angles       = np.asarray(angles)
        FF_val_nodih = np.asarray(FF_val_nodih)
        FF_ei_nodih  = np.asarray(FF_ei_nodih)
        FF_vdw_nodih = np.asarray(FF_vdw_nodih)
        FF_tot_nodih = np.asarray(FF_tot_nodih)
        
        FF_val_wdih = np.asarray(FF_val_wdih)
        FF_ei_wdih  = np.asarray(FF_ei_wdih)
        FF_vdw_wdih = np.asarray(FF_vdw_wdih)
        FF_tot_wdih = np.asarray(FF_tot_wdih)
        
        FF_poly   = np.asarray(FF_poly)
        ai_energy = np.asarray(ai_energy)
        
    
    # Plot FF and AI data and compare with fit
    fig,ax = pt.subplots(1,2,sharex=True,sharey=True,figsize=(25,10))
    ax[0].plot(angles,centre(FF_tot_nodih+FF_poly), label='tot_new_ff')
    ax[0].plot(angles,centre(ai_energy), label='AI')
    ax[0].set_xlabel(r'$\psi$ ($^\circ$)')
    ax[0].set_ylabel('E (kJ/mol)')
    ax[0].legend(bbox_to_anchor=(1.0,.5), loc=6)
    ax[0].set_title('Energy barrier w polysix term')

    ax[1].plot(angles,centre(ai_energy-FF_tot_nodih), label='diff')
    ax[1].plot(angles,centre(FF_poly), label='polysix')
    ax[1].set_xlabel(r'$\psi$ ($^\circ$)')
    ax[1].set_ylabel('E (kJ/mol)')
    ax[1].legend(bbox_to_anchor=(1.0,.5), loc=6)
    ax[1].set_title('Energy difference vs polysix')
    
    pt.show()
    
def full_plot(pr,name,sinfo,aid,ffd,save_png=False):
    print(name)
    odd = True # will select correct poly function
    
    vdw_names = ['pair_mm3','pair_lj','pair_mm3cap']
    if len(ffd.table_nodih)>0:
        vdw_name = [name for name in vdw_names if name in list(ffd.table_nodih.columns)]
    else:
        vdw_name = [name for name in vdw_names if name in list(ffd.table_wdih.columns)]
    assert len(vdw_name)==1
    vdw_name = vdw_name[0]
    
    
    # FF data
    if len(ffd.table_nodih)>0:
        angles       = ffd.table_nodih.sort_values(by=['angle']).angle
        FF_val_nodih = ffd.table_nodih.sort_values(by=['angle']).valence/kjmol
        FF_ei_nodih  = ffd.table_nodih.sort_values(by=['angle']).pair_ei/kjmol
        FF_vdw_nodih = ffd.table_nodih.sort_values(by=['angle'])[vdw_name]/kjmol
        FF_tot_nodih = ffd.table_nodih.sort_values(by=['angle']).e_tot/kjmol
        
    else:
        angles       = ffd.table_wdih.sort_values(by=['angle']).angle
    
    FF_val_wdih = ffd.table_wdih.sort_values(by=['angle']).valence/kjmol
    FF_ei_wdih  = ffd.table_wdih.sort_values(by=['angle']).pair_ei/kjmol
    FF_vdw_wdih = ffd.table_wdih.sort_values(by=['angle'])[vdw_name]/kjmol
    FF_tot_wdih = ffd.table_wdih.sort_values(by=['angle']).e_tot/kjmol
    
    poly_angles  = ffd.table_polysix.sort_values(by=['angle']).angle
    FF_poly     = ffd.table_polysix.sort_values(by=['angle']).e_tot/kjmol
    
    # AI data
    if aid is not None:
        ai_angles   = aid.sort_values(by=['angle']).angle
        ai_energy   = aid.sort_values(by=['angle']).E/kjmol
    
    try:
        assert np.all(np.isclose(np.asarray(ai_angles),np.asarray(angles), atol=1e-3))
        assert np.all(np.isclose(np.asarray(poly_angles),np.asarray(angles), atol=1e-3))
    except AssertionError:
        print(np.asarray(ai_angles)-np.asarray(angles))
        print(np.asarray(poly_angles)-np.asarray(angles))
        raise AssertionError

    
    # If angle range is limited to 90 degrees mirror the behaviour to 180
    def mirror(array, angle=False):
        if angle:
            return np.hstack((array, 180-array[:-1][::-1]))
        else:
            return np.hstack((array, array[:-1][::-1]))
        
    def mirror60(array, angle=False):
        if angle:
            return np.hstack((array, 120-array[:-1][::-1], 120+array[1:]))
        else:
            return np.hstack((array, array[:-1][::-1], array[1:]))
        
    if np.isclose(np.max(angles),90,atol=1e-4):
        print('Mirroring')
        odd = False
        angles       = mirror(angles,angle=True)
        if len(ffd.table_nodih)>0:
            FF_val_nodih = mirror(FF_val_nodih)
            FF_ei_nodih  = mirror(FF_ei_nodih)
            FF_vdw_nodih = mirror(FF_vdw_nodih)
            FF_tot_nodih = mirror(FF_tot_nodih)
        
        FF_val_wdih = mirror(FF_val_wdih)
        FF_ei_wdih  = mirror(FF_ei_wdih)
        FF_vdw_wdih = mirror(FF_vdw_wdih)
        FF_tot_wdih = mirror(FF_tot_wdih)
        FF_poly     = mirror(FF_poly)
        
        if aid is not None:
            ai_energy   = mirror(ai_energy)
            
    elif np.isclose(np.max(angles),60,atol=1e-3):
        print('Mirroring - 60')
        angles       = mirror60(angles,angle=True)
        if len(ffd.table_nodih)>0:
            FF_val_nodih = mirror60(FF_val_nodih)
            FF_ei_nodih  = mirror60(FF_ei_nodih)
            FF_vdw_nodih = mirror60(FF_vdw_nodih)
            FF_tot_nodih = mirror60(FF_tot_nodih)
        
        FF_val_wdih = mirror60(FF_val_wdih)
        FF_ei_wdih  = mirror60(FF_ei_wdih)
        FF_vdw_wdih = mirror60(FF_vdw_wdih)
        FF_tot_wdih = mirror60(FF_tot_wdih)
        FF_poly     = mirror60(FF_poly)
        
        if aid is not None:
            ai_energy   = mirror60(ai_energy)           
    else:
        angles      = np.asarray(angles)
        
        if len(ffd.table_nodih)>0:
            FF_val_nodih = np.asarray(FF_val_nodih)
            FF_ei_nodih  = np.asarray(FF_ei_nodih)
            FF_vdw_nodih = np.asarray(FF_vdw_nodih)
            FF_tot_nodih = np.asarray(FF_tot_nodih)
        
        FF_val_wdih = np.asarray(FF_val_wdih)
        FF_ei_wdih  = np.asarray(FF_ei_wdih)
        FF_vdw_wdih = np.asarray(FF_vdw_wdih)
        FF_tot_wdih = np.asarray(FF_tot_wdih)
        FF_poly     = np.asarray(FF_poly)
        
        if aid is not None:
            ai_energy   = np.asarray(ai_energy)
            
    # Plot data
    fig,ax = pt.subplots(2,sharex=True,sharey=True,figsize=(15,10))
    ax[0].plot(angles,centre(FF_val_wdih), label='valence')
    ax[0].plot(angles,centre(FF_ei_wdih), label='ei')
    ax[0].plot(angles,centre(FF_vdw_wdih), label='vdw')
    ax[0].plot(angles,centre(FF_tot_wdih), ls='--', label='tot')
    ax[0].plot(angles,centre(ai_energy), label='AI')
    ax[0].set_xlabel(r'$\psi$ ($^\circ$)')
    ax[0].set_ylabel('E (kJ/mol)')
    ax[0].legend(bbox_to_anchor=(1.0,.5), loc=6)
    ax[0].set_title('Energy barrier - old FF')
    ax[1].plot(angles,centre(FF_val_nodih + FF_poly), label='valence')
    ax[1].plot(angles,centre(FF_ei_nodih), label='ei')
    ax[1].plot(angles,centre(FF_vdw_nodih), label='vdw')
    ax[1].plot(angles,centre(FF_tot_nodih + FF_poly), ls='--', label='tot')
    ax[1].plot(angles,centre(ai_energy), label='AI')
    ax[1].set_xlabel(r'$\psi$ ($^\circ$)')
    ax[1].set_ylabel('E (kJ/mol)')
    ax[1].legend(bbox_to_anchor=(1.0,.5), loc=6)
    ax[1].set_title('Energy barrier - new FF')
    
    if save_png:
        path = pr.root_path +  pr.project_path.split('/')[0] + '/input_files/' + '/'.join(pr.project_path.split('/')[1:]) + sinfo.ff.block_name + '/fits/'
        pathlib.Path(path).mkdir(parents=True, exist_ok=True) # make directory if it does not exist
        pt.savefig(path+'final_{}.pdf'.format(name))
    pt.show()

### Execution

#### PART 1

In [ ]:
# Perform the scans
for name,sinfo in sinfos_triazine.items():
    if not name in gic_triazine:
        print(name)
        yaff_scan_job(sinfo.pr, 'scan_'+name+'_polysix', sinfo, sinfo.ff.ffpars_polysix)

In [ ]:
# Perform the scans
for name,sinfo in sinfos_triazine.items():
    if name in gic_triazine:
        pr_sub_g16 = Project(sinfo.get_sub_project_name(name))
        structures = [pr_sub_g16.load('job_{}'.format(n)).get_structure() for n,_ in enumerate(np.arange(sinfo.range_min, sinfo.range_max+sinfo.range_step, sinfo.range_step))]
        yaff_scan_structures_job(sinfo.pr, 'scan_'+name+'_polysix', sinfo, sinfo.ff.ffpars_polysix, structures)

In [ ]:
# table_polysix will be calculated in 6: Testing polysix terms 
for name,sinfo in sinfos_triazine.items():
    grid = np.arange(sinfo.range_min, sinfo.range_max+sinfo.range_step, sinfo.range_step)
    ff_triazine_datas[name].table_polysix = get_table_polysix(name,sinfo,grid)

In [ ]:
for name,sinfo in sinfos_triazine.items():
    full_plot(sinfo.pr,name,sinfo,ai_data_triazine[name],ff_triazine_datas[name],save_png=True)

#### PART 2

In [ ]:
# Perform the scans - no GIC
for name,sinfo in sinfos_ext.items():
    print(name)
    name = name.translate({ord(c): None for c in '!@#$(),/-'})
    yaff_scan_job(sinfo.pr, 'scan_'+name+'_polysix', sinfo, sinfo.ff.ffpars_polysix)

In [ ]:
# table_polysix will be calculated in 6: Testing polysix terms 
for name,sinfo in sinfos_ext.items():
    grid = np.arange(sinfo.range_min, sinfo.range_max+sinfo.range_step, sinfo.range_step)
    ff_ext_datas[name].table_polysix = get_table_polysix(name,sinfo,grid)

In [ ]:
for name,sinfo in sinfos_ext.items():
    full_plot(sinfo.pr,name,sinfo,ai_data_ext[name],ff_ext_datas[name],save_png=True)

## Testing new QuickFF force field

### Functions

In [ ]:
import difflib

def overlap_ffpars(ffp1,ffp2):
    # Compare two force field parameter files and return the overlapping terms
    # If second argument is a list, perform iteratively
    assert isinstance(ffp1,str)
    if isinstance(ffp2,list):
        ref_ffp = copy.copy(ffp1)
        for ffp in ffp2:
            ref_ffp = overlap_ffpars(ref_ffp,ffp)
        return ref_ffp
    
    elif isinstance(ffp2,str):
        new_ffp = ''
        diff = difflib.ndiff(ffp1.splitlines(keepends=True),ffp2.splitlines(keepends=True))
        for s in diff:
            if s[0]==' ': # no changes
                new_ffp += s[2:]
        return new_ffp
    
    else:
        raise ValueError
        
def extend_ffpars(ffp1,ffp2):
    # Compare two force field parameter files and return both the overlap and the extensions
    # If second argument is a list, perform iteratively
    assert isinstance(ffp1,str)
    if isinstance(ffp2,list):
        ref_ffp = copy.copy(ffp1)
        for ffp in ffp2:
            ref_ffp = extend_ffpars(ref_ffp,ffp)
        return ref_ffp
    
    elif isinstance(ffp2,str):
        new_ffp = ''
        diff = difflib.ndiff(ffp1.splitlines(keepends=True),ffp2.splitlines(keepends=True))
        for s in diff:
            if not s[0]=='?': # eye guides in diff
                new_ffp += s[2:]
        return new_ffp
    
    else:
        raise ValueError

def compare_structure_old(name,sinfo,old,new,ai,save_png=False):
    structures = {'old':old, 'new':new, 'ai':ai}
    compare_structures(name,sinfo.pr,sinfo.ff,structures,save_png=False)

### Execution

#### PART 1

In [ ]:
# Perform the optimization
for key,val in sinfo_triazine_groups.items():
    print('{}: {}'.format(key,','.join(list(val.keys()))))

    val_keys = list(val.keys())
    print(val_keys)
    sinfos = [val[k] for k in val_keys]  
    sinfo = sinfos[0]

    ffpars = sinfo.ff.ffpars
    ffpars_nodih = overlap_ffpars(sinfo.ff.ffpars_nodih,[sinfo.ff.ffpars_nodih for sinfo in sinfos[1:]])
    ffpars_polysix = extend_ffpars(sinfo.ff.ffpars_polysix,[sinfo.ff.ffpars_polysix for sinfo in sinfos[1:]])

    pr_sub = Project(sinfo.get_sub_project_name(key,suffix='ffopt'))

    yaff_opt_job(pr_sub, 'opt_old_ff', sinfo.ff.structure, sinfo.ff, ffpars)
    yaff_opt_job(pr_sub, 'opt_new_ff', sinfo.ff.structure, sinfo.ff, ffpars_nodih + ffpars_polysix)

In [ ]:
# Compare structures
for key,val in sinfo_triazine_groups.items():
    val_keys = list(val.keys())
    print(val_keys)
    sinfos = [val[k] for k in val_keys]  
    sinfo = sinfos[0]

    pr_sub = Project(sinfo.get_sub_project_name(key,suffix='ffopt'))
    compare_structure_old(key,sinfo,pr_sub.load('opt_old_ff').get_structure(),pr_sub.load('opt_new_ff').get_structure(),sinfo.ff.structure,save_png=True)

#### PART 2

In [ ]:
# Perform the optimization
for key,val in sinfo_ext_groups.items():
    print('{}: {}'.format(key,','.join(list(val.keys()))))

    val_keys = list(val.keys())
    print(val_keys)
    sinfos = [val[k] for k in val_keys]  
    sinfo = sinfos[0]

    ffpars = sinfo.ff.ffpars
    ffpars_nodih = overlap_ffpars(sinfo.ff.ffpars_nodih,[sinfo.ff.ffpars_nodih for sinfo in sinfos[1:]])
    ffpars_polysix = extend_ffpars(sinfo.ff.ffpars_polysix,[sinfo.ff.ffpars_polysix for sinfo in sinfos[1:]])

    pr_sub = Project(sinfo.get_sub_project_name(key,suffix='ffopt'))

    yaff_opt_job(pr_sub, 'opt_old_ff', sinfo.ff.structure, sinfo.ff, ffpars)
    yaff_opt_job(pr_sub, 'opt_new_ff', sinfo.ff.structure, sinfo.ff, ffpars_nodih + ffpars_polysix)

In [ ]:
# Compare structures
for key,val in sinfo_ext_groups.items():
    val_keys = list(val.keys())
    print(val_keys)
    sinfos = [val[k] for k in val_keys]  
    sinfo = sinfos[0]

    pr_sub = Project(sinfo.get_sub_project_name(key,suffix='ffopt'))
    compare_structure_old(key,sinfo,pr_sub.load('opt_old_ff').get_structure(),pr_sub.load('opt_new_ff').get_structure(),sinfo.ff.structure,save_png=True)

## Writing clean QuickFF force field

### Functions

In [ ]:
# return array with terms and their values
def read_pars_cov(fn):
    terms = ["BONDHARM:PARS", "BENDAHARM:PARS", "BENDCLIN:PARS", "BENDCOS:PARS", "TORSION:PARS", "TORSCPOLYSIX:PARS", "OOPDIST:PARS", "SQOOPDIST:PARS", "Cross:PARS"]
    
    # Assume covalent parameters always have the correct header
    
    bondharm  = []
    bendharm  = []
    bendclin  = []
    bendcos   = []
    torsion   = []
    torscpoly = []
    oopdist   = []
    sqoopdist = []
    cross     = []
        
    with open(fn, 'r') as f:
        text = f.read()
        split = text.split()

        for i in range(len(split)):
            if(split[i] == terms[0]):
                bondharm.append(add_term(split, [i+1, i+2], [i+3], [i+4]))    
            elif(split[i] == terms[1]):
                bendharm.append(add_term(split, [i+1, i+2, i+3], [i+4], [i+5]))
            elif(split[i] == terms[2]):
                bendclin.append(add_term(split, [i+1, i+2, i+3], [i+4]))    
            elif(split[i] == terms[3]):
                bendcos.append(add_term(split, [i+1, i+2, i+3], [i+5], [i+6], index_mult=i+4))    
            elif(split[i] == terms[4]):
                torsion.append(add_term(split, [i+1, i+2, i+3, i+4], [i+6], [i+7], index_mult=i+5))
            elif(split[i] == terms[5]):
                torscpoly.append(add_term(split, [i+1, i+2, i+3, i+4], [i+5, i+6, i+7, i+8, i+9, i+10]))    
            elif(split[i] == terms[6]):
                oopdist.append(add_term(split, [i+1, i+2, i+3, i+4], [i+5], [i+6]))
            elif(split[i] == terms[7]):
                sqoopdist.append(add_term(split, [i+1, i+2, i+3, i+4], [i+5], [i+6]))
            elif(split[i] == terms[8]):
                cross.append(add_term(split, [i+1, i+2, i+3], [i+4, i+5, i+6], [i+7, i+8, i+9]))             
    
    dic = []
    dic.append(bondharm)
    dic.append(bendharm)
    dic.append(bendclin)
    dic.append(bendcos)
    dic.append(torsion)
    dic.append(torscpoly)
    dic.append(oopdist)
    dic.append(sqoopdist)
    dic.append(cross)
    
    return dic
   
# return array with terms and their values
def read_pars_ei(fn,units,scales,dielectric):
    terms_ei = ["FIXQ:ATOM", "FIXQ:BOND"]
    terms_header = ["FIXQ:UNIT","FIXQ:SCALE","FIXQ:DIELECTRIC"]
    
    atomcharge = []
    bondcharge = []
        
    with open(fn, 'r') as f:
        text = f.read()
        split = text.split()
        
        for i in range(len(split)):
            if(split[i] == terms_ei[0]):
                atomcharge.append(add_term(split, [i+1], [i+2, i+3]))    
            elif(split[i] == terms_ei[1]):
                bondcharge.append(add_term(split, [i+1, i+2], [i+3]))
            elif(split[i] == terms_header[0]):
                units[split[i+1]] = split[i+2]
            elif(split[i] == terms_header[1]):
                scales[int(split[i+1])] = float(split[i+2]) 
            elif(split[i] == terms_header[1]):
                dielectric = float(split[i+1])    
            
            
    dic = []
    dic.append(atomcharge)
    dic.append(bondcharge)
    return dic


# return array with terms and their values
def read_pars_vdw(fn,units,scales):
    vdws = {'mm3':[],'mm3cap':[],'lj':[]}
    terms_vdw = ["MM3:PARS", "MM3CAP:PARS", "LJ:PARS"]
    terms_header = ["MM3:UNIT","MM3CAP:UNIT","LJ:UNIT","MM3:SCALE","MM3CAP:SCALE","LJ:SCALE"]
    
    mm3 = []
    mm3cap = []
    lj = []
        
    with open(fn, 'r') as f:
        text = f.read()
        split = text.split()
        
        for i in range(len(split)):
            if(split[i] == terms_vdw[0]):
                vdws['mm3'].append(add_term(split, [i+1], [i+2, i+3, i+4]))
            elif(split[i] == terms_vdw[1]):
                vdws['mm3cap'].append(add_term(split, [i+1], [i+2, i+3, i+4]))
            elif(split[i] == terms_vdw[2]):
                vdws['lj'].append(add_term(split, [i+1], [i+2, i+3]))
            elif(split[i] in terms_header[0:3]):
                units[split[i+1]] = split[i+2]
            elif(split[i] in terms_header[3:]):
                scales[int(split[i+1])] = float(split[i+2])       
    
    lengths = [len(v) for v in vdws.values()]
    assert max(lengths)==sum(lengths) # check that only one of them is used
    
    key = list(vdws.keys())[np.argmax(lengths)]
    return vdws[key],key

In [ ]:
def print_separate_pars_file(fns, blocks, max_length, ei_units, ei_scales, dielectric, vdw_units, vdw_scales, vdw_key='mm3'):
    cov_values = blocks[0]
    ei_values = blocks[1]
    vdw_values = blocks[2]
    
    with open(fns[0], 'w') as f:            
        
        # Write covalent part
        f.write("# BONDHARM\n#---------\nBONDHARM:UNIT  K kjmol/A**2\nBONDHARM:UNIT  R0 A\n\n")
        for pair in cov_values[0]:
            f.write("BONDHARM:PARS  ")
            f.write("{:>{w}}  {:>{w}}  {: 1.10e}  {: 1.10e}".format(pair[0][0], pair[0][1], float(pair[1][0]), float(pair[2][0]), w=max_length))
            f.write('\n')
        
        f.write("\n\n")
        f.write("# BENDAHARM\n#----------\nBENDAHARM:UNIT  K kjmol/rad**2\nBENDAHARM:UNIT  THETA0 deg\n\n")
        for pair in cov_values[1]:
            f.write("BENDAHARM:PARS  ")
            f.write("{:>{w}}  {:>{w}}  {:>{w}}  {: 1.10e}  {: 1.10e}".format(pair[0][0], pair[0][1], pair[0][2], float(pair[1][0]), float(pair[2][0]), w=max_length))
            f.write('\n')
        
        if cov_values[2]:  #check whether there is anything in this list   
            f.write("\n\n")
            f.write("# BENDCLIN\n#----------\nBENDCLIN:UNIT  A kjmol\n\n")
            for pair in cov_values[2]:
                f.write("BENDCLIN:PARS  ")
                f.write("{:>{w}}  {:>{w}}  {:>{w}}  {: 1.10e}".format(pair[0][0], pair[0][1], pair[0][2], float(pair[1][0]), w=max_length))
                f.write('\n')
                
        if cov_values[3]:  #check whether there is anything in this list   
            f.write("\n\n")
            f.write("# BENDCOS\n#----------\nBENDCOS:UNIT  A kjmol\nBENDCOS:UNIT PHI0 deg\n\n")
            for pair in cov_values[3]:
                f.write("BENDCOS:PARS  ")
                f.write("{:>{w}}  {:>{w}}  {:>{w}} {} {: 1.10e} {: 1.10e}".format(pair[0][0], pair[0][1], pair[0][2], int(pair[3]), float(pair[1][0]), float(pair[2][0]), w=max_length))
                f.write('\n')
        
        f.write("\n\n")
        f.write("# TORSION\n#--------\nTORSION:UNIT  A kjmol\nTORSION:UNIT  PHI0 deg\n\n")
        
        for pair in cov_values[4]:
            f.write("TORSION:PARS  ")
            f.write("{:>{w}}  {:>{w}}  {:>{w}}  {:>{w}}  {} {: 1.10e}  {: 1.10e}".format(pair[0][0], pair[0][1], pair[0][2], pair[0][3], int(pair[3]), float(pair[1][0]), float(pair[2][0]), w=max_length))
            f.write('\n')
        
        if cov_values[5]:  #check whether there is anything in this list
            f.write("\n\n")
            f.write("# TORSCPOLYSIX\n#--------\nTORSCPOLYSIX:UNIT  C1 kjmol\nTORSCPOLYSIX:UNIT  C2 kjmol\nTORSCPOLYSIX:UNIT  C3 kjmol\nTORSCPOLYSIX:UNIT  C4 kjmol\nTORSCPOLYSIX:UNIT  C5 kjmol\nTORSCPOLYSIX:UNIT  C6 kjmol\n\n")

            for pair in cov_values[5]:
                f.write("TORSCPOLYSIX:PARS  ")
                f.write("{:>{w}}  {:>{w}}  {:>{w}}  {:>{w}}  {: 1.10e}  {: 1.10e}  {: 1.10e}  {: 1.10e}  {: 1.10e}  {: 1.10e}".format(pair[0][0], pair[0][1], pair[0][2], pair[0][3], float(pair[1][0]), float(pair[1][1]), float(pair[1][2]), float(pair[1][3]), float(pair[1][4]), float(pair[1][5]), w=max_length))
                f.write('\n')
        
        if cov_values[6]:  #check whether there is anything in this list
            f.write("\n\n")
            f.write("# OOPDIST\n#--------\nOOPDIST:UNIT  K kjmol/A**2\nOOPDIST:UNIT  D0 A\n\n")

            for pair in cov_values[6]:
                f.write("OOPDIST:PARS  ")
                f.write("{:>{w}}  {:>{w}}  {:>{w}}  {:>{w}}  {: 1.10e}  {: 1.10e}".format(pair[0][0], pair[0][1], pair[0][2], pair[0][3], float(pair[1][0]), float(pair[2][0]), w=max_length))
                f.write('\n')
        
        if cov_values[7]:  #check whether there is anything in this list
            f.write("\n\n")
            f.write("# SQOOPDIST\n#--------\nSQOOPDIST:UNIT  K kjmol/A**4\nSQOOPDIST:UNIT  D0 A**2\n\n")

            for pair in cov_values[7]:
                f.write("SQOOPDIST:PARS  ")
                f.write("{:>{w}}  {:>{w}}  {:>{w}}  {:>{w}}  {: 1.10e}  {: 1.10e}".format(pair[0][0], pair[0][1], pair[0][2], pair[0][3], float(pair[1][0]), float(pair[2][0]), w=max_length))
                f.write('\n')

        f.write("\n\n")
        f.write("# Cross\n#------\nCross:UNIT  KSS kjmol/angstrom**2\nCross:UNIT  KBS0 kjmol/(angstrom*rad)\nCross:UNIT  KBS1 kjmol/(angstrom*rad)\nCross:UNIT  R0 angstrom\nCross:UNIT  R1 angstrom\nCross:UNIT  THETA0 deg\n\n")

        for pair in cov_values[8]:
            f.write("CROSS:PARS  ")
            f.write("{:>{w}}  {:>{w}}  {:>{w}}  {: 1.10e}  {: 1.10e}  {: 1.10e}  {: 1.10e}  {: 1.10e}  {: 1.10e}".format(pair[0][0], pair[0][1], pair[0][2], float(pair[1][0]), float(pair[1][1]), float(pair[1][2]), float(pair[2][0]), float(pair[2][1]), float(pair[2][2]), w=max_length))
            f.write('\n')
        
        f.write("\n\n")
        
    with open(fns[1], 'w') as f:  
        
        # Write ei part
        f.write("#Fixed charges\n#---------------\n\n")
        f.write("".join(["FIXQ:UNIT {} {}\n".format(k,v) for k,v in ei_units.items()]) + "".join(["FIXQ:SCALE {} {}\n".format(k,v) for k,v in ei_scales.items()]) + "FIXQ:DIELECTRIC {}\n\n".format(dielectric))
        f.write("# Atomic parameters\n# ----------------------------------------------------\n# KEY        label  Q_0A              R_A\n# ----------------------------------------------------\n")
        for pair in ei_values[0]:
            f.write("FIXQ:ATOM ")
            f.write("{:>{w}}  {: 1.10f}  {: 1.10f}".format(pair[0][0], float(pair[1][0]), float(pair[1][1]), w=max_length))
            f.write('\n')
        
        f.write("# Bond parameters\n# ----------------------------------------------------\n# KEY         label0   label1           P_AB\n# ----------------------------------------------------\n")
        for pair in ei_values[1]:
            f.write("FIXQ:BOND  ")
            f.write("{:>{w}}  {:>{w}}  {: 1.10f}".format(pair[0][0], pair[0][1], float(pair[1][0]), w=max_length))
            f.write('\n')
            
    with open(fns[2], 'w') as f:
        
        # Write VDW part
        f.write("# van der Waals\n#---------------\n# The following mathemetical form is supported:\n#  - MM3(CAP):   EPSILON*(1.84e5*exp(-12*r/SIGMA)-2.25*(SIGMA/r)^6)\n#  - LJ:    4.0*EPSILON*((SIGMA/r)^12 - (SIGMA/r)^6)\n\n")
        if vdw_key=='mm3':
            f.write("".join(["MM3:UNIT {} {}\n".format(k,v) for k,v in vdw_units.items()]) + "".join(["MM3:SCALE {} {}\n".format(k,v) for k,v in vdw_scales.items()]))
            f.write("\n# ---------------------------------------------\n# KEY      ffatype    SIGMA  EPSILON  ONLYPAULI\n# ---------------------------------------------\n")
            for pair in vdw_values:
                f.write("MM3:PARS ")
                f.write("{:>{w}}  {:1.4f}  {:1.4f}  {}".format(pair[0][0], float(pair[1][0]), float(pair[1][1]), int(pair[1][2]), w=max_length))
                f.write('\n')
        elif vdw_key=='mm3cap':
            f.write("".join(["MM3CAP:UNIT {} {}\n".format(k,v) for k,v in vdw_units.items()]) + "".join(["MM3CAP:SCALE {} {}\n".format(k,v) for k,v in vdw_scales.items()]))
            f.write("\n# ---------------------------------------------\n# KEY      ffatype    SIGMA  EPSILON  ONLYPAULI\n# ---------------------------------------------\n")
            for pair in vdw_values:
                f.write("MM3CAP:PARS ")
                f.write("{:>{w}}  {:1.4f}  {:1.4f}  {}".format(pair[0][0], float(pair[1][0]), float(pair[1][1]), int(pair[1][2]), w=max_length))
                f.write('\n')
        elif vdw_key=='lj':
            f.write("".join(["LJ:UNIT {} {}\n".format(k,v) for k,v in vdw_units.items()]) + "".join(["LJ:SCALE {} {}\n".format(k,v) for k,v in vdw_scales.items()]))
            f.write("# ---------------------------------------------\n# KEY      ffatype    SIGMA  EPSILON\n# ---------------------------------------------\n")
            for pair in vdw_values:
                f.write("LJ:PARS ")
                f.write("{:>{w}}  {:1.4f}  {:1.4f}".format(pair[0][0], float(pair[1][0]), float(pair[1][1]), w=max_length))
                f.write('\n')   
        else:
            raise ValueError('Key not recognized')

In [ ]:
def write_separate_ffpars(sinfo_groups,name_function=None):
    """
        Create separated ffpars files in its contributions (cov,ei,vdw)
        Unique ffpars will be created per key in sinfo_groups
        if for a single block there are multiple keys
    """
    block_names = []
    for key,val in sinfo_groups.items():
        sinfos = [val[k] for k in val.keys()]  
        sinfo = sinfos[0]
        block_names.append(sinfo.ff.block_name)
        
    c = Counter(block_names)
    
    for key,val in sinfo_groups.items():
        print('{}: {}'.format(key,','.join(list(val.keys()))))

        val_keys = list(val.keys())
        #print(val_keys)
        sinfos = [val[k] for k in val_keys]  
        sinfo = sinfos[0]
        
        pr_sub_name = sinfo.get_sub_project_name(key,suffix='ffopt')
        print(pr_sub_name)

        pr_sub = Project(pr_sub_name)
        opt_job = pr_sub.load('opt_new_ff') 

        # Get the parameter file of the opt job
        fn_ffpars = os.path.join(opt_job.working_directory, 'pars.txt')

        # Read the separate ffpars contributions
        ei_units,ei_scales,vdw_units,vdw_scales = {},{},{},{}
        dielectric = 1.0

        cov_block = read_pars_cov(fn_ffpars)
        ei_block = read_pars_ei(fn_ffpars,ei_units,ei_scales,dielectric)
        vdw_block, vdw_key = read_pars_vdw(fn_ffpars,vdw_units,vdw_scales)

        # Get max atype length
        if len(ei_block[0]) > 0:
            max_length = max([len(pair[0][0]) for pair in ei_block[0]])
        elif len(vdw_block) > 0:
            max_length = max([len(pair[0][0]) for pair in vdw_block])
        else:
            raise ValueError
        #print(max_length)

        # Write pars file
        path = sinfo.pr.root_path +  sinfo.pr.project_path.split('/')[0] + '/input_files/' + '/'.join(sinfo.pr.project_path.split('/')[1:]) + sinfo.ff.block_name + '/new_ffpars/'
        
        if c[sinfo.ff.block_name]>1:
            assert name_function is not None
            path = path + '{}/'.format(name_function(key))
        pathlib.Path(path).mkdir(parents=True, exist_ok=True) # make directory if it does not exist

        fns = [path + 'pars_{}.txt'.format(bl) for bl in ['cov','ei','vdw']]
        print_separate_pars_file(fns, [cov_block,ei_block,vdw_block], max_length, ei_units, ei_scales, dielectric, vdw_units, vdw_scales, vdw_key=vdw_key)
        

### Executions

In [ ]:
write_separate_ffpars(sinfo_triazine_groups)

In [ ]:
write_separate_ffpars(sinfo_ext_groups)